In [31]:
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics import f1_score

In [8]:
train_df=pd.read_csv("/data/aig/public/wuchunsheng/NLP/competition/data/cnews/train_set.csv",sep="\t",nrows=15000)

tfidf=TfidfVectorizer(ngram_range=(1,3),max_features=3000)
train_test=tfidf.fit_transform(train_df["text"])


In [14]:
dtrain=xgb.DMatrix(train_test[:10000],label=train_df["label"][:10000])

In [15]:
dtest=xgb.DMatrix(train_test[10000:])

In [27]:
param={
    "max_depth":6,
    "eta":0.5,
    "eval_metric":"merror",
    "silent":1,
    "objective":"multi:softmax",
    "num_class":14
      }
evallist=[(dtrain,"train")]
num_round=15
bst=xgb.train(param,dtrain,num_round,evallist)

[15:02:09] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.12850
[1]	train-merror:0.08060
[2]	train-merror:0.05660
[3]	train-merror:0.04160
[4]	train-merror:0.03040
[5]	train-merror:0.02200
[6]	train-merror:0.01490
[7]	train-merror:0.01240
[8]	train-merror:0.00840
[9]	train-merror:0.00540
[10]	train-merror:0.00340
[11]	train-merror:0.00260
[12]	train-merror:0.00200
[13]	train-merror:0.00130
[14]	train-merror:0.00110


In [32]:
val_pred=bst.predict(dtest)
val_pred
print(f1_score(train_df["label"].values[10000:],val_pred,average="macro"))

0.8578152416704026



### gridsearch

In [37]:
from sklearn.model_selection import GridSearchCV

In [42]:
param_grid={
    "max_depth":[5,10,20],
    "learning_rate":[0.01,0.02,0.05,0.1,0.15],
    "n_estimators":[50,100,300],
    "min_child_weight":[0,2,5,10,20],
    "max_delta_step":[0,0.2,0.6,1,2],
    "subsample":[0.6,0.8,0.95],
    "colsample_bytree":[0.5,0.8,0.99],
    
}

xlf=xgb.XGBClassifier()
gsearch=GridSearchCV(xlf,param_grid=param_grid,cv=5)



/data/aig/public/wuchunsheng/install/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: fit() got an unexpected keyword argument 'label'

  FitFailedWarning)


TypeError: fit() got an unexpected keyword argument 'label'

In [ ]:
gsearch.fit(train_test[:10000],train_df["label"][:10000])